# RNN을 이용한 SMS SPAM 분류
- 데이터 다운로드 캐글

In [1]:
!apt-get update -qq
!apt-get install fonts-nanum* -qq

Selecting previously unselected package fonts-nanum.
(Reading database ... 146442 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [2]:
# 설치뒤 font-manager rebuild
import matplotlib.font_manager as fm
fm._rebuild()

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

seed= 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0]


Saving spam.csv to spam.csv


In [3]:
import pandas as pd
df = pd.read_csv(filename, encoding='latin1')
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [4]:
del df['Unnamed: 2']
del df['Unnamed: 3']
del df['Unnamed: 4']
df['v1'] = df['v1'].replace(['ham', 'spam'], [0,1])
df.head(3)

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   int64 
 1   v2      5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


In [6]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [9]:
df['v2'].nunique()

5169

In [10]:
df = df.drop_duplicates('v2', keep='first')

In [11]:
df['v1'].value_counts()

0    4516
1     653
Name: v1, dtype: int64

In [12]:
X_data = df['v2'].values
y_data = df['v1'].values
print('SMS 본문의 개수:', len(X_data))
print('레이블의 개수:', len(y_data))

SMS 본문의 개수: 5169
레이블의 개수: 5169


In [13]:
t = Tokenizer()
t.fit_on_texts(X_data)
sequences = t.texts_to_sequences(X_data)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 8921


In [14]:
X_data = sequences

### 전체 데이터셋의 길이를 임의의 숫자(100)에 맞춤


In [15]:
data = pad_sequences(X_data, maxlen=100)
data.shape

(5169, 100)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data, y_data, stratify=y_data, test_size=0.2, random_state=seed
)

In [17]:
X_train.shape, X_test.shape

((4135, 100), (1034, 100))

### 모델 정의/설정/학습

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [21]:
model = Sequential()
model.add(Embedding(vocab_size, 32)) # 임베딩 벡터의 차원은 32
model.add(SimpleRNN(32))             # RNN 셀의 hidden_size는 32
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          285472    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 287,585
Trainable params: 287,585
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(loss='binary_crossentropy', 
              optimizer='rmsprop', metrics=['accuracy'])

In [23]:
history = model.fit(X_train, y_train, epochs=10, 
                    batch_size=60, validation_split=0.2)

Epoch 1/10
56/56 [==============================] - 6s 64ms/step - loss: 0.3762 - accuracy: 0.8750 - val_loss: 0.1172 - val_accuracy: 0.9770
Epoch 2/10
56/56 [==============================] - 3s 61ms/step - loss: 0.1015 - accuracy: 0.9747 - val_loss: 0.0620 - val_accuracy: 0.9855
Epoch 3/10
56/56 [==============================] - 3s 60ms/step - loss: 0.0518 - accuracy: 0.9878 - val_loss: 0.0544 - val_accuracy: 0.9855
Epoch 4/10
56/56 [==============================] - 3s 60ms/step - loss: 0.0359 - accuracy: 0.9893 - val_loss: 0.0855 - val_accuracy: 0.9758
Epoch 5/10
56/56 [==============================] - 3s 62ms/step - loss: 0.0360 - accuracy: 0.9908 - val_loss: 0.0757 - val_accuracy: 0.9782
Epoch 6/10
56/56 [==============================] - 3s 61ms/step - loss: 0.0124 - accuracy: 0.9972 - val_loss: 0.0689 - val_accuracy: 0.9819
Epoch 7/10
56/56 [==============================] - 4s 63ms/step - loss: 0.0065 - accuracy: 0.9991 - val_loss: 0.0841 - val_accuracy: 0.9770
Epoch 8/10
56

In [24]:
acc = model.evaluate(X_test, y_test, verbose=2)[1]
print(f'Accuracy: {acc:.4f}')

33/33 - 0s - loss: 0.0755 - accuracy: 0.9816
Accuracy: 0.9816
